**8. Calculating metrics on players level**

In the subsequent notebooks, we make an attempt to drill down further on the player level to understand how different LCBs tend to prefer different regions for making progressive passes and how much threat do they add in each region, based on their dominant foot.

The following tasks have been taken into account in this notebook:
1. Compute passing metrics (similar to match level as before) and group them by individual players

# Imports 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Ellipse
import seaborn as sns
from math import *
import matplotlib.pylab as pyl
import pickle
import swifter
import warnings
import plotly.express as px
from itertools import chain
import scipy.stats as sps
from tqdm import tqdm
from unidecode import unidecode
import re
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd
import itertools


In [2]:
import warnings
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)

In [3]:
#pd.set_option('max_colwidth', 999)
pd.set_option('display.max_columns', 1000)
pd.set_option("display.max_rows", 3000)

# Data 

In [6]:
# Loading clusters with VAEP values as a list of dataframes
cluster_names = [
    'rll', 'rlll', 'rlr', 'rrl', 'rrll', 'rrlll', 'rrlr', 'rrr', 'rrrl',
    'rrrll', 'rrrlr', 'rrrr', 'rrrrl'
]
df_clusters = list()
for name in cluster_names:
    df_clusters.append(
        pd.read_pickle(
            f'../data/clusters/clusters_vaep/cluster_{name}.pkl'
        ))

In [7]:
# Reseting the indexes of dfs in df_clusters
for df in df_clusters:
    df.reset_index(drop=True, inplace = True)

# Creating Player-Metrics Dataset

In [8]:
df_clusters[4].head()

,wyId,team,team_defense,RB,R_CB,L_CB,LB,RCB,CB,LCB,RWB,LWB,backline,match,gameweek,teamsData,dateutc,venue,referees,score,footedness,temp,RB_pass,RB_accpass,RB_accpassloc,RB_inaccpassloc,RB_accvaep,RB_inaccvaep,RB_accoff,RB_inaccoff,RB_accdef,RB_inaccdef,R_CB_pass,R_CB_accpass,R_CB_accpassloc,R_CB_inaccpassloc,R_CB_accvaep,R_CB_inaccvaep,R_CB_accoff,R_CB_inaccoff,R_CB_accdef,R_CB_inaccdef,L_CB_pass,L_CB_accpass,L_CB_accpassloc,L_CB_inaccpassloc,L_CB_accvaep,L_CB_inaccvaep,L_CB_accoff,L_CB_inaccoff,L_CB_accdef,L_CB_inaccdef,LB_pass,LB_accpass,LB_accpassloc,LB_inaccpassloc,LB_accvaep,LB_inaccvaep,LB_accoff,LB_inaccoff,LB_accdef,LB_inaccdef
0,2500090,Crystal Palace,"[AaronWanBissaka, JamesTomkins, MamadouSakho, ...",AaronWanBissaka,JamesTomkins,MamadouSakho,PatrickvanAanholt,NaN,NaN,NaN,NaN,NaN,4.0,Crystal Palace-West Bromwich Albion,38,"{'1628': {'scoreET': 0, 'coachId': 8357, 'side...",2018-05-13 14:00:00,Selhurst Park,"[{'refereeId': 381851, 'role': 'referee'}, {'r...",2 - 0,right-right-left-left,2500090Crystal Palace,58,53,"[[[60.32, 11.56], [70.72, 8.84]], [[55.12, 11....","[[[86.32, 22.44], [87.36, 25.84]], [[98.8, 10....","[0.0027392818592488766, -0.0019667267333716154...","[-0.02877645380795002, -0.01624043844640255, -...","[0.002814173698425293, -0.0016991370357573032,...","[-0.027683690190315247, -0.01549511682242155, ...","[-7.489195559173822e-05, -0.000267589697614312...","[-0.0010927643161267042, -0.000745321158319711...",43,39,"[[[20.8, 14.96], [31.2, 11.56]], [[30.16, 25.1...","[[[22.88, 25.84], [88.4, 8.16]], [[47.84, 21.0...","[0.0004819142632186413, 0.0018559431191533804,...","[0.005953557789325714, -0.004551937337964773, ...","[0.0025302842259407043, 0.0013152025640010834,...","[0.0027480272110551596, -0.00383941363543272, ...","[-0.002048369962722063, 0.000540740555152297, ...","[0.0032055305782705545, -0.000712523702532053,...",61,53,"[[[38.48, 46.24], [40.56, 27.2]], [[31.2, 50.3...","[[[40.56, 58.48], [79.04, 51.0]], [[42.64, 55....","[-0.00042911351192742586, -0.00118101888801902...","[-0.0015874714590609074, -0.004032387398183346...","[-0.00017807912081480026, -0.00152375176548957...","[-0.0005808919668197632, -0.001907807309180498...","[-0.0002510343911126256, 0.0003427328774705529...","[-0.0010065794922411442, -0.002124580089002847...",49,45,"[[[35.36, 63.24], [54.08, 57.8]], [[35.36, 65....","[[[89.44, 53.04], [95.68, 44.88]], [[38.48, 55...","[0.003728308714926243, -0.00019248295575380325...","[-0.02945968694984913, -0.0020599353592842817,...","[0.0037858709692955017, -0.0002007205039262771...","[-0.0272841714322567, 3.7888530641794205e-05, ...","[-5.756225436925888e-05, 8.237548172473907e-06...","[-0.002175515051931143, -0.002097823889926076,..."
1,2500095,Manchester City,"[Danilo, JohnStones, AymericLaporte, FabianDelph]",Danilo,JohnStones,AymericLaporte,FabianDelph,NaN,NaN,NaN,NaN,NaN,4.0,Southampton-Manchester City,38,"{'1625': {'scoreET': 0, 'coachId': 267136, 'si...",2018-05-13 14:00:00,St. Mary's Stadium,"[{'refereeId': 385911, 'role': 'referee'}, {'r...",0 - 1,right-right-left-left,2500095Manchester City,76,70,"[[[59.28, 4.08], [68.64, 26.52]], [[58.24, 9.5...","[[[38.48, 10.88], [40.56, 6.8]], [[35.36, 10.2...","[0.0038701544981449842, 0.0037723213899880648,...","[-0.001479479717090726, -0.002215859480202198,...","[0.004012984223663807, 0.003661242313683033, 0...","[7.367925718426704e-06, -0.000706125982105732,...","[-0.00014282972551882267, 0.000111079076305031...","[-0.0014868476428091526, -0.001509733498096466...",73,71,"[[[35.36, 34.0], [82.16, 61.88]], [[30.16, 19....","[[[33.28, 31.96], [38.48, 32.64]], [[8.32, 57....","[0.010264113545417786, -0.0018642968498170376,...","[0.004142566584050655, -0.004952588118612766]","[0.010572408325970173, -0.0010678647086024284,...","[0.0007246871246024966, -0.0004069982096552849]","[-0.00030829478055238724, -0.00079643214121460...","[0.003417879343032837, -0.004545589908957481]",77,67,"[[[39.52, 24.48], [30.16, 19.04]], [[27.04, 52...","[[[29.12, 47.

In [9]:
# Function to reorganise the match level metrics dataframe to player level
def player_metrics_func(l):
    player_metrics = pd.DataFrame(columns=[
        'player_name',
        'team',
        'backline_footedness',
        'backline',
        'position',
        'wyId',
        'passes',
        'accpass',
        'accpassloc',
        'inaccpassloc',
        'accpassvaep',
        'inaccpassvaep',
        'accpassoff',
        'inaccpassoff',
        'accpassdef',
        'inaccpassdef'
        ])
    pos = [
        'RB',
        'R_CB',
        'L_CB',
        'LB',
        'RWB',
        'RCB',
        'CB',
        'LCB',
        'LWB',
        ]
    for df in l:
        for i in tqdm(range(len(df))):
            if df['backline'][i] == 4:
                s1 = list()
                for p in pos[0:4]:
                    p1 = p + '_'
                    s = [[df[p][i]], [df['team'][i]], [df['footedness'
                         ][i]], [df['backline'][i]], [p],[df['wyId'][i]], df.loc[i,
                         list(df.columns[df.columns.str.startswith(p1)])].values,]
                    s = list(itertools.chain(*s))
                    s1.append(s)
                s1 = np.vstack(s1)
                player_metrics = pd.concat([player_metrics,
                        pd.DataFrame(s1,
                        columns=player_metrics.columns)], axis=0)
            elif df['backline'][i] == 3:

                s1 = list()
                for p in pos[5:8]:
                    p1 = p + '_'
                    s = [[df[p][i]], [df['team'][i]], [df['footedness'
                         ][i]], [df['backline'][i]], [p],[df['wyId'][i]], df.loc[i,
                         list(df.columns[df.columns.str.startswith(p1)])].values]
                    s = list(itertools.chain(*s))
                    s1.append(s)
                s1 = np.vstack(s1)
                player_metrics = pd.concat([player_metrics,
                        pd.DataFrame(s1,
                        columns=player_metrics.columns)], axis=0)
            elif df['backline'][i] == 5:

                s1 = list()
                for p in pos[4:]:
                    p1 = p + '_'
                    s = [[df[p][i]], [df['team'][i]], [df['footedness'
                         ][i]], [df['backline'][i]], [p],[df['wyId'][i]], df.loc[i,
                         list(df.columns[df.columns.str.startswith(p1)])].values]
                    s = list(itertools.chain(*s))
                    s1.append(s)
                s1 = np.vstack(s1)
                player_metrics = pd.concat([player_metrics,
                        pd.DataFrame(s1,
                        columns=player_metrics.columns)], axis=0)

    return player_metrics


In [10]:
player_metrics = player_metrics_func(df_clusters)
print(player_metrics.shape)
player_metrics.reset_index(drop=True, inplace=True)


(2885, 16)


In [11]:
player_metrics.tail(20)

,player_name,team,backline_footedness,backline,position,wyId,passes,accpass,accpassloc,inaccpassloc,accpassvaep,inaccpassvaep,accpassoff,inaccpassoff,accpassdef,inaccpassdef
2865,EzequielSchelotto,Brighton and Hove Albion,right-right-right-right-left,5,RWB,2499950,39,32,"[[[15.6, 6.12], [11.44, 18.36]], [[66.56, 17.6...","[[[101.92, 16.32], [95.68, 53.04]], [[92.56, 6...","[-0.0027250798884779215, 0.008709050714969635,...","[-0.022368621081113815, -0.0026102035772055387...","[-0.0021365147549659014, 0.008279751054942608,...","[-0.021815089508891106, -0.0029721278697252274...","[-0.0005885651335120201, 0.0004292994271963835...","[-0.000553530640900135, 0.0003619242925196886,..."
2866,ConnorGoldson,Brighton and Hove Albion,right-right-right-right-left,5,RCB,2499950,37,29,"[[[20.8, 13.6], [15.6, 34.0]], [[31.2, 20.4], ...","[[[11.44, 18.36], [46.8, 17.68]], [[32.24, 19....","[-0.0016521192155778408, -0.000637517310678958...","[-0.005109456367790699, -0.005683832801878452,...","[-0.001610324252396822, -0.0005284212529659271...","[4.69256192445755e-05, -0.0028726253658533096,...","[-4.179496318101883e-05, -0.000109096057713031...","[-0.0051563819870352745, -0.002811207436025142..."
2867,ShaneDuffy,Brighton and Hove Albion,right-right-right-right-left,5,CB,2499950,31,28,"[[[21.84, 47.6], [30.16, 36.72]], [[20.8, 33.3...","[[[20.8, 59.84], [28.08, 54.4]], [[26.0, 27.88...","[0.00019070273265242577, 0.0017127888277173042...","[-0.009465785697102547, -0.016421709209680557,...","[1.1037569493055344e-05, 0.0001666424795985221...","[-0.00037744222208857536, 9.625079110264778e-0...","[0.00017966516315937042, 0.001546146348118782,...","[-0.009088343009352684, -0.016517959535121918,..."
2868,LewisDunk,Brighton and Hove Albion,right-right-right-right-left,5,LCB,2499950,37,27,"[[[26.0, 63.24], [30.16, 64.6]], [[18.72, 37.4...","[[[31.2, 44.88], [43.68, 48.96]], [[28.08, 59....","[-0.0010328111238777637, 0.0074929846450686455...","[-0.007338287308812141, -0.0001143945846706628...","[-0.0005701703485101461, 0.0010665850713849068...","[-0.002070433460175991, 6.990809924900532e-05,...","[-0.0004626407753676176, 0.006426399573683739,...","[-0.00526785384863615, -0.0001843026839196682,..."
2869,MarkusSuttner,Brighton and Hove Albion,right-right-right-right-left,5,LWB,2499950,25,19,"[[[41.6, 51.0], [39.52, 46.92]], [[61.36, 48.9...","[[[101.92, 61.88], [89.44, 26.52]], [[91.52, 6...","[-0.001984398579224944, 0.0032602078281342983,...","[-0.012535693123936653, -0.024366769939661026,...","[-0.0011985022574663162, 0.0031371312215924263...","[-0.010827042162418365, -0.021044034510850906,...","[-0.0007858963217586279, 0.0001230766065418720...","[-0.0017086504958570004, -0.003322735195979476..."
2870,KyleNaughton,Swansea City,right-right-right-right-left,5,RWB,2499957,39,27,"[[[30.16, 6.12], [14.56, 8.84]], [[34.32, 2.04...","[[[32.24, 4.76], [35.36, 6.12]], [[33.28, 6.8]...","[-0.00030438019894063473, 0.001873926026746630...","[-0.00579429604113102, -0.010925043374300003, ...","[-0.00043917191214859486, 0.001523824874311685...","[-0.002266661264002323, -0.002798699773848057,...","[0.00013479171320796013, 0.0003501011524349451...","[-0.0035276347771286964, -0.008126343600451946..."
2871,MikevanderHoorn,Swansea City,right-right-right-right-left,5,RCB,2499957,27,18,"[[[30.16, 21.76], [40.56, 14.96]], [[30.16, 16...","[[[60.32, 5.44], [70.72, 21.76]], [[21.84, 12....","[2.3149419575929642e-05, 0.01246498804539442, ...","[-0.0032844608649611473, 0.0024564226623624563...","[0.00012397533282637596, 0.0022160641383379698...","[-0.001406063325703144, 0.000289801275357604, ...","[-0.00010082591325044632, 0.010248923674225807...","[-0.0018783975392580032, 0.0021666213870048523..."
2872,FedericoFernandez,Swansea City,right-right-right-right-left,5,CB,2499957,17,14,"[[[31.2, 33.32], [30.16, 21.76]], [[41.6, 16.3...","[[[52.0, 32.64], [71.76, 38.76]], [[27.04, 33....","[0.0012881478760391474, 0.004821080714464188, ...","[-0.0016094238962978125, 0.010347407311201096,...

In [12]:
# Function to add number of values of match level metrics per match
def match_idx(row):
    acc_match_idx = [len(row['accpassloc'])]
    inacc_match_idx = [len(row['inaccpassloc'])]
    return pd.Series([acc_match_idx,inacc_match_idx])
    

In [13]:
player_metrics[['acc_match_idx','inacc_match_idx']] = player_metrics.apply(lambda row: match_idx(row), axis=1 )

In [14]:
player_metrics.loc[player_metrics['player_name']=='AdamSmith']

,player_name,team,backline_footedness,backline,position,wyId,passes,accpass,accpassloc,inaccpassloc,accpassvaep,inaccpassvaep,accpassoff,inaccpassoff,accpassdef,inaccpassdef,acc_match_idx,inacc_match_idx
665,AdamSmith,AFC Bournemouth,right-right-left-left,4,RB,2499889,46,39,"[[[28.08, 4.76], [40.56, 20.4]], [[31.2, 13.6]...","[[[55.12, 4.08], [62.4, 6.8]], [[35.36, 4.08],...","[0.0007454273290932178, -0.002337034558877349,...","[-0.008189908228814602, 0.0007318942807614803,...","[0.0013971277512609959, -0.0004184735007584095...","[-0.005722924135625362, 0.0007222630083560944,...","[-0.000651700422167778, -0.0019185610581189394...","[-0.0024669840931892395, 9.631272405385971e-06...",[39],[7]
669,AdamSmith,AFC Bournemouth,right-right-left-left,4,RB,2499884,45,39,"[[[14.56, 6.12], [9.36, 15.64]], [[48.88, 24.4...","[[[43.68, 20.4], [66.56, 1.36]], [[92.56, 3.4]...","[-0.0020024231635034084, -0.000500710681080818...","[0.001774220960214734, -0.002873945515602827, ...","[-0.0003442731685936451, -0.000260645523667335...","[-0.00013712630607187748, -0.00231535919010639...","[-0.0016581499949097633, -0.000240065157413482...","[0.0019113472662866116, -0.000558586441911757,...",[39],[6]
721,AdamSmith,AFC Bournemouth,right-right-left-left,4,RB,2499859,31,26,"[[[31.2, 8.16], [18.72, 10.88]], [[38.48, 8.84...","[[[43.68, 8.16], [62.4, 8.84]], [[31.2, 6.12],...","[0.00097347900737077, -0.0011119181290268898, ...","[-0.0016525862738490105, -0.000662904232740402...","[0.0002457159571349621, -0.00067894347012043, ...","[-0.0016500959172844887, 0.0003468878567218780...","[0.0007277630502358079, -0.0004329746589064598...","[-2.4903565645217896e-06, -0.00100979208946228...",[26],[5]
741,AdamSmith,AFC Bournemouth,right-right-left-left,4,RB,2499846,60,53,"[[[63.44, 11.56], [69.68, 21.76]], [[74.88, 10...","[[[27.04, 5.44], [39.52, 8.16]], [[29.12, 2.72...","[0.005353913176804781, 0.030311232432723045, 0...","[-0.005121732130646706, -0.0018177926540374756...","[0.005366073921322823, 0.030763717368245125, 0...","[-0.0014389478601515293, 0.0001220800913870334...","[-1.216074451804161e-05, -0.000452485051937401...","[-0.0036827842704951763, -0.001939872745424509...",[53],[7]
821,AdamSmith,AFC Bournemouth,right-right-left-left,4,RB,2499805,43,30,"[[[63.44, 1.36], [46.8, 6.8]], [[28.08, 5.44],...","[[[43.68, 1.36], [47.84, 4.08]], [[24.96, 6.12...","[-0.0027234056033194065, 0.008349435403943062,...","[-0.007041573990136385, -0.005091824568808079,...","[-0.0026750662364065647, 0.0028343996964395046...","[-0.0034799561835825443, -0.002193832071498036...","[-4.8339483328163624e-05, 0.00551503524184227,...","[-0.0035616178065538406, -0.002897992264479398...",[30],[13]
845,AdamSmith,AFC Bournemouth,right-right-left-left,4,RB,2499779,65,54,"[[[55.12, 8.16], [57.2, 32.64]], [[70.72, 12.9...","[[[67.6, 4.08], [81.12, 1.36]], [[30.16, 4.76]...","[0.0004147776635363698, 0.05057172104716301, 0...","[-0.009886120446026325, 0.009931659325957298, ...","[-0.00018300116062164307, 0.052904173731803894...","[-0.00873370748013258, 0.002039430197328329, -...","[0.0005977788241580129, -0.0023324524518102407...","[-0.0011524130823090672, 0.007892228662967682,...",[54],[11]
857,AdamSmith,AFC Bournemouth,right-right-left-left,4,RB,2499772,40,35,"[[[79.04, 18.36], [101.92, 10.2]], [[91.52, 3....","[[[101.92, 1.36], [101.92, 2.04]], [[73.84, 14...","[-0.0027333241887390614, -0.005029985215514898...","[-0.029243893921375275, -0.012092892080545425,...","[-0.0030440762639045715, -0.005688074976205826...","[-0.02838980033993721, -0.01062268391251564, 0...","[0.0003107520751655102, 0.0006580897606909275,...","[-0.0008540935814380646, -0.001470208400860428...",[35],[5]
873,AdamSmith,AFC Bournemouth,right-right-left-left,4,RB,2499759,56,54,"[[[33.28, 1.36], [19.76, 7.48]], [[66.56, 8.84...","[[[80.08, 8.84], [95.68, 22.44]], [[38.48, 17....","[-0.00034849834628403187, 0.003234653500840068...","[-0.003870188258588314, -0.0021765632554888725]","[-0.0004601788241416216, 0.00330387055873870

In [15]:
# Function to extract dominant foot information
def extract_foot(a,b):
    pos1 = ['RB', 'R_CB', 'L_CB', 'LB']
    pos2 = ['RWB', 'RCB', 'CB', 'LCB', 'LWB']
    a = a.split('-')
    if(len(a)==4):
        return a[pos1.index(b)]
    elif(len(a)==5):
        return a[pos2.index(b)]
    elif(len(a)==3):
        return a[pos2.index(b) - 1]

In [16]:
player_metrics['footedness'] = player_metrics.apply(lambda x: extract_foot(x.backline_footedness, x.position), axis=1)
player_metrics.drop(columns = ['backline_footedness'], inplace=True)

In [17]:
player_metrics.head(5)

,player_name,team,backline,position,wyId,passes,accpass,accpassloc,inaccpassloc,accpassvaep,inaccpassvaep,accpassoff,inaccpassoff,accpassdef,inaccpassdef,acc_match_idx,inacc_match_idx,footedness
0,SteveCook,AFC Bournemouth,3,RCB,2500089,40,27,"[[[64.48, 19.72], [64.48, 4.76]], [[46.8, 11.5...","[[[27.04, 4.76], [39.52, 4.76]], [[34.32, 8.16...","[-0.0017563197761774063, 0.0015748164150863886...","[-0.006077437195926905, -0.0012522567994892597...","[-0.002439117059111595, 0.001446905080229044, ...","[-0.0024494631215929985, 0.0004311646334826946...","[0.0006827972829341888, 0.00012791133485734463...","[-0.003627974074333906, -0.0016834214329719543...",[27],[13],right
1,NathanAke,AFC Bournemouth,3,CB,2500089,30,24,"[[[48.88, 36.72], [39.52, 54.4]], [[6.24, 3.4]...","[[[28.08, 22.44], [55.12, 22.44]], [[31.2, 25....","[-0.0009504464687779546, 0.013750978745520115,...","[0.001986460294574499, -0.009018474258482456, ...","[-0.0012554381974041462, 0.002747779479250312,...","[0.0005576326511800289, 0.000248798169195652, ...","[0.0003049917286261916, 0.01100319903343916, 0...","[0.0014288276433944702, -0.009267272427678108,...",[24],[6],left
2,TyroneMings,AFC Bournemouth,3,LCB,2500089,45,38,"[[[32.24, 47.6], [68.64, 48.96]], [[39.52, 54....","[[[33.28, 56.44], [65.52, 48.28]], [[19.76, 36...","[0.010723656043410301, 0.00023811450228095055,...","[-0.0038944093976169825, -0.024236343801021576...","[0.010727065615355968, 9.276997298002243e-05, ...","[-0.0014773723669350147, -0.001994863385334611...","[-3.409339115023613e-06, 0.0001453445293009281...","[-0.0024170370306819677, -0.02224148064851761,...",[38],[7],left
3,DeclanRice,West Ham United,3,RCB,2500098,35,26,"[[[41.6, 18.36], [43.68, 33.32]], [[48.88, 14....","[[[28.08, 12.92], [68.64, 28.56]], [[36.4, 5.4...","[-0.000301242689602077, 0.007525372318923473, ...","[-0.002428452018648386, -0.004736741539090872,...","[-0.00043192552402615547, 0.007223010063171387...","[-0.001414981670677662, -0.003226976376026869,...","[0.00013068283442407846, 0.000302362022921443,...","[-0.001013470347970724, -0.001509765163064003,...",[26],[9],right
4,AngeloOgbonna,West Ham United,3,CB,2500098,25,24,"[[[24.96, 31.96], [40.56, 17.0]], [[38.48, 44....","[[[33.28, 35.36], [75.92, 46.24]]]","[0.0016360594891011715, 0.0007009587716311216,...",[-0.0014785041566938162],"[0.0011539123952388763, 0.0006793192587792873,...",[0.00024306168779730797],"[0.00048214709386229515, 2.1639512851834297e-0...",[-0.0017215658444911242],[24],[1],left


In [18]:
# Replace some of the misspelled player names that occured due to unidecode conversion
player_metrics = player_metrics.replace({'player_name':{'RamiroFunesMori': 'JoseRamiroFunesMori',
                'KurtZouma': 'KurtHappyZouma',
                'Danilo': 'DaniloLuizdaSilva',
                'CesarAzpilicueta': 'CesarAzpilicuetaTanco',
                'EzequielSchelotto': 'MatiasEzequielSchelotto',
                'GaetanBong': 'GaetanBongSongo',
                'HectorBellerin': 'HectorBellerinMoruno',
                'AhmedHegazi': 'AhmedHegazy',
                'JamaalLascelles': 'JamalLascelles',
                'AngelRangel': 'AngelRangelZaragoza',
                'Zanka': 'MathiasJattahNjieJorgensen',
                'EricBailly': 'EricBertrandBailly',
                'MarcosRojo': 'FaustinoMarcosAlbertoRojo',
                'AngeloOgbonna': 'AngeloObinzeOgbonna',
                'DavinsonSanchez': 'DavinsonSanchezMina',
                'JavierManquillo': 'JavierManquilloGaitan',
                'TommySmith': 'TomSmith',
                'Bruno': 'BrunoSaltorGrau',
                'JosephGomez': 'JoeGomez',
                'AlbertoMoreno':'AlbertoMorenoPerez',
                'LuisAntonioValencia':'LuisAntonioValenciaMosquera',
                'NicolasOtamendi':'NicolasHernanOtamendi',
                'NachoMonreal':'IgnacioMonrealEraso',
                'CedricSoares':'CedricRicardoAlvesSoares',
                'JoelMatip':'JoelAndreJobMatip',
                'MiguelBritos':'MiguelAngelBritosCabrera',
                'VictorLindelof':'VictorNilssonLindelof',
                'JamesCollins':'JamesMichaelCollins',
                'CucoMartina':'RhuendlyMartina',
                'DavidLuiz':'DavidLuizMoreiraMarinho',
                'ChancelMbemba':'ChancelMbembaMangulu',
                'PabloZabaleta':'PabloJavierZabaletaGirod',
                'KikoFemenia':'FranciscoFemeniaFar',
                'JoseFonte':'JoseMigueldaRochaFonte',
                'JesusGamez':'JesusGamezDuarte'
                }})


In [19]:
# Import team contribution data
team_contri = pd.read_pickle("../data/vaep/value_sums.pkl")
team_contri.head()

,offsum_regionwise,vaepsum_regionwise,team_name,match_id
0,"[0.008872812148183584, 0.3449347550049424, 0.8...","[0.005850694258697331, 0.3173894425854087, 0.7...",Arsenal,2499719
1,"[2.1650491072796285, 0.6611172530101612, 0.403...","[2.0625359127297997, 0.6439085232559592, 0.334...",Leicester City,2499719
2,"[0.4810514331329614, 0.9121930369874462, 0.624...","[0.32630410650745034, 0.8620082812849432, 0.57...",Manchester City,2499720
3,"[-0.022033099317923188, 0.1337635952513665, 0....","[-0.06919947441201657, 0.09574798995163292, 0....",Brighton and Hove Albion,2499720
4,"[0.0856203967705369, 0.6818049850407988, 0.794...","[0.12690815760288388, 0.5602604085579515, 0.72...",Chelsea,2499721


In [20]:
# Merge team contribution data with player metrics
player_metrics_contri = pd.merge(player_metrics,team_contri,left_on=['wyId','team'], right_on=['match_id','team_name'])

In [21]:
player_metrics_contri.drop(['team_name','match_id'],axis=1,inplace=True)

In [22]:
# Group the data by individual players, their team, role and dominant foot
df = player_metrics_contri.groupby([
    'player_name', 'team', 'position', 'footedness'
])['passes', 'accpass', 'accpassloc', 'inaccpassloc', 'accpassvaep',
   'inaccpassvaep', 'accpassoff', 'inaccpassoff', 'accpassdef', 'inaccpassdef',
    'acc_match_idx', 'inacc_match_idx', 'offsum_regionwise',
   'vaepsum_regionwise'].sum().reset_index()

In [23]:
player_metrics_contri['matches_played'] = pd.Series(
    np.ones(len(player_metrics_contri)))
df1 = player_metrics_contri.groupby(
    ['player_name', 'team', 'position',
     'footedness'])['matches_played'].sum().reset_index()
df['matches_played'] = df1.matches_played

In [24]:
df.head()

,player_name,team,position,footedness,passes,accpass,accpassloc,inaccpassloc,accpassvaep,inaccpassvaep,accpassoff,inaccpassoff,accpassdef,inaccpassdef,acc_match_idx,inacc_match_idx,offsum_regionwise,vaepsum_regionwise,matches_played
0,AaronCresswell,West Ham United,LB,left,275,203,"[[[11.44, 47.6], [10.4, 48.96]], [[21.84, 58.4...","[[[22.88, 65.28], [31.2, 63.24]], [[24.96, 61....","[-0.0060007767751812935, 0.014951384626328945,...","[-0.005001368001103401, 0.0014556727837771177,...","[-0.00111871468834579, 0.0030969707295298576, ...","[-0.0018319590017199516, 0.0011951092164963484...","[-0.00488206185400486, 0.011854413896799088, 0...","[-0.003169409232214093, 0.00026056356728076935...","[38, 31, 30, 34, 15, 17, 38]","[17, 13, 4, 7, 9, 10, 12]","[-0.057076123310253024, 0.4220066689886153, -0...","[-0.14396203577052802, 0.430011440301314, -0.7...",7.0
1,AaronCresswell,West Ham United,LCB,left,796,644,"[[[27.04, 51.68], [55.12, 64.6]], [[65.52, 61....","[[[62.4, 51.0], [78.0, 49.64]], [[79.04, 61.2]...","[0.0010694738011807203, 0.002371369395405054, ...","[-0.00956201646476984, -0.007710381411015987, ...","[0.0018120664171874523, 0.002711281180381775, ...","[-0.006647953763604164, -0.006097717210650444,...","[-0.0007425926742143929, -0.000339911784976720...","[-0.002914062701165676, -0.001612664433196187,...","[36, 34, 32, 42, 31, 38, 55, 48, 20, 17, 26, 1...","[15, 6, 8, 10, 3, 10, 5, 13, 4, 7, 3, 7, 12, 7...","[0.12159526557661593, 0.7864310317672789, 0.53...","[0.12645460478961468, 0.7321038860827684, 0.47...",20.0
2,AaronWanBissaka,Crystal Palace,RB,right,226,165,"[[[60.32, 11.56], [70.72, 8.84]], [[55.12, 11....","[[[86.32, 22.44], [87.36, 25.84]], [[98.8, 10....","[0.0027392818592488766, -0.0019667267333716154...","[-0.02877645380795002, -0.01624043844640255, -...","[0.002814173698425293, -0.0016991370357573032,...","[-0.027683690190315247, -0.01549511682242155, ...","[-7.489195559173822e-05, -0.000267589697614312...","[-0.0010927643161267042, -0.000745321158319711...","[53, 30, 10, 26, 26, 15, 5]","[5, 5, 10, 15, 9, 6, 11]","[0.22176160640083253, 0.5252915103919804, 0.88...","[0.2210844011278823, 0.5201125951716676, 0.865...",7.0
3,AdamSmith,AFC Bournemouth,LB,right,146,120,"[[[15.6, 59.84], [22.88, 54.4]], [[19.76, 61.2...","[[[67.6, 13.6], [74.88, 33.32]], [[44.72, 2.72...","[-0.002463837619870901, -0.0001035716850310564...","[-0.007104361429810524, 1.648860052227974e-05,...","[-0.005229263566434383, 0.0009387347381561995,...","[-0.005188643001019955, 0.0009281975217163563,...","[0.0027654259465634823, -0.0010423064231872559...","[-0.0019157183123752475, -0.000911708921194076...","[46, 35, 39]","[5, 13, 8]","[0.0014031482860445976, 0.9950951430946589, 0....","[0.0035754673881456256, 0.9505251653026789, 0....",3.0
4,AdamSmith,AFC Bournemouth,RB,right,451,387,"[[[28.08, 4.76], [40.56, 20.4]], [[31.2, 13.6]...","[[[55.12, 4.08], [62.4, 6.8]], [[35.36, 4.08],...","[0.0007454273290932178, -0.002337034558877349,...","[-0.008189908228814602, 0.0007318942807614803,...","[0.0013971277512609959, -0.0004184735007584095...","[-0.005722924135625362, 0.0007222630083560944,...","[-0.000651700422167778, -0.0019185610581189394...","[-0.0024669840931892395, 9.631272405385971e-06...","[39, 39, 26, 53, 30, 54, 35, 54, 29, 28]","[7, 6, 5, 7, 13, 11, 5, 2, 4, 4]","[0.12391482037492096, 0.3068442572839558, 0.14...","[0.09303268906660378, 0.20946494350209832, 0.0...",10.0


In [26]:
df.to_pickle('../data/defender_clusters/defender_clustering_dataset.pkl')